## lets repeat what we did before

In [2]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
Table = soup.find('table')
Tabel_split = Table.tbody.text.split('\n')


lst1 =[] ;lst2 =[];lst3 =[] 
temp_val = 0
rows = 1
while (len(Tabel_split)>temp_val):
    lst1.append(Tabel_split[5*rows+1])
    lst2.append(Tabel_split[5*rows+2])
    lst3.append(Tabel_split[5*rows+3])
    temp_val = 5*rows+6
    rows += 1

df=pd.DataFrame(list(zip(lst1,lst2, lst3)), columns =['PostCode', 'Borough', 'Neighbourhood']) 
df.head()


print('Number of Boroughs with Not Assigned values:',df[df['Borough'].str.contains('Not assigned')].count()[0])
print('Number of Neighbourhoods with Not Assigned values:',df[df['Neighbourhood'].str.contains('Not assigned')].count()[0])
df=df[df['Borough']!='Not assigned']
print()
print('The Neighbourhoods with Not Assigned value:')
print(df[df['Neighbourhood'].str.contains('Not assigned')])
print()
df.replace(to_replace='Not assigned',value="Queen's Park", inplace=True)
print('Now, the number of Boroughs with Not Assigned values:',df[df['Borough'].str.contains('Not assigned')].count()[0])
print('Now, the number of Neighbourhoods with Not Assigned values is:',df[df['Neighbourhood'].str.contains('Not assigned')].count()[0])


df_final = df.groupby(['PostCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
print('Total number of Boroughs: {}'.format(len(df_final['Borough'])))
df_final.head()

Number of Boroughs with Not Assigned values: 77
Number of Neighbourhoods with Not Assigned values: 78

The Neighbourhoods with Not Assigned value:
  PostCode       Borough Neighbourhood
8      M7A  Queen's Park  Not assigned

Now, the number of Boroughs with Not Assigned values: 0
Now, the number of Neighbourhoods with Not Assigned values is: 0
Total number of Boroughs: 103


,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

latitude=[]
longitude=[]
print('Looking for coordinats for each postal code ...')
for indx,postcode in enumerate(list(df_final['PostCode'])):
    address = '{} Toronto, Ontario, Canada'.format(postcode)
    print("Progress: {} out of {}".format(indx,df_final['PostCode'].shape[0]-1), end="\r")
    
    try:
        geolocator = Nominatim(user_agent="tn_explorer")
        location = geolocator.geocode(address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        
    except:
        latitude.append(None)
        longitude.append(None)
        
print('\nDone!')

Looking for coordinats for each postal code ...


In [ ]:
df_final['Latitude'] = latitude
df_final['Longitude'] = longitude
print('Number of missing coordinate values is %d which is bascially %.2f%% of all the values.' %(df_final['Latitude'].isna().sum(),(df_final['Latitude'].isna().sum()/df_final['Latitude'].shape[0]*100)))